In [1]:
%load_ext autoreload
%autoreload 2
from __future__ import absolute_import

In [6]:
from extract_data import *
from finbeat_calc import *
from plot_track import *
from plot_accel import *
from check_plots import *
import pywt
import pylab
import numpy as np

# Take position data from hardcoded folder, calc. velocities, accels
tracklist = extract_data()

# Calculate finbeat peaks and troughs

finbeats, finbeat_byP, finbeat_byT = finbeat_calc(tracklist.keys(), tracklist)

# Extract finbeat periods, amplitudes, and subsequent accel

# Plot things if desired
# plot_track(['sometrial'], tracklist)
# plot_accel(['sometrial'], tracklist)

# """If you want to plot the peaks and troughs on the position data,
# uncomment section in finbeat_calc"""

C:\Anaconda3\lib\site-packages\numpy\lib\polynomial.py:595: RankWarning: Polyfit may be poorly conditioned
  warnings.warn(msg, RankWarning)
C:\Anaconda3\lib\site-packages\numpy\lib\polynomial.py:595: RankWarning: Polyfit may be poorly conditioned
  warnings.warn(msg, RankWarning)
C:\Anaconda3\lib\site-packages\numpy\lib\polynomial.py:595: RankWarning: Polyfit may be poorly conditioned
  warnings.warn(msg, RankWarning)
C:\Anaconda3\lib\site-packages\numpy\lib\polynomial.py:595: RankWarning: Polyfit may be poorly conditioned
  warnings.warn(msg, RankWarning)
C:\Anaconda3\lib\site-packages\numpy\lib\polynomial.py:595: RankWarning: Polyfit may be poorly conditioned
  warnings.warn(msg, RankWarning)
C:\Anaconda3\lib\site-packages\numpy\lib\polynomial.py:595: RankWarning: Polyfit may be poorly conditioned
  warnings.warn(msg, RankWarning)
C:\Anaconda3\lib\site-packages\numpy\lib\polynomial.py:595: RankWarning: Polyfit may be poorly conditioned
  warnings.warn(msg, RankWarning)
C:\Anaconda3\

In [3]:
finbeats


({'BTrout1S01': {'behavior': 'S', 'fb_data':      time type       ypos
   0   0.016    T  22.345278
   1   0.196    P  23.467802
   2   0.340    T  22.227815
   3   0.478    P  23.280979
   4   0.608    T  22.229566
   5   0.734    P  23.389224
   6   0.860    T  22.106848
   7   0.986    P  23.423069
   8   1.108    T  22.223980
   9   1.222    P  23.195017
   10  1.340    T  22.081515
   11  1.470    P  23.319335
   12  1.586    T  22.272340
   13  1.712    P  23.357984, 'fish': 'BTrout1', 'trial_name': 'BTrout1S01'},
  'BTrout1S02': {'behavior': 'S', 'fb_data':      time type       ypos
   0   0.074    T  23.041690
   1   0.218    P  24.332466
   2   0.354    T  23.113533
   3   0.498    P  24.458195
   4   0.628    T  23.281492
   5   0.764    P  24.568397
   6   0.892    T  23.349227
   7   1.032    P  24.736694
   8   1.162    T  23.534150
   9   1.288    P  24.576155
   10  1.428    T  23.202312
   11  1.562    P  24.280976
   12  1.712    T  22.875556
   13  1.848    P  24.0512

In [8]:
finbeat_byP

{'BTrout1S01':      time type       ypos  endtime  period  nxttrough  amplitude
 1   0.196    P  23.467802    0.478   0.282  22.227815   1.239987
 3   0.478    P  23.280979    0.734   0.256  22.229566   1.051413
 5   0.734    P  23.389224    0.986   0.252  22.106848   1.282376
 7   0.986    P  23.423069    1.222   0.236  22.223980   1.199089
 9   1.222    P  23.195017    1.470   0.248  22.081515   1.113503
 11  1.470    P  23.319335    1.712   0.242  22.272340   1.046996
 13  1.712    P  23.357984      NaN     NaN        NaN        NaN,
 'BTrout1S02':      time type       ypos  endtime  period  nxttrough  amplitude
 1   0.218    P  24.332466    0.498   0.280  23.113533   1.218933
 3   0.498    P  24.458195    0.764   0.266  23.281492   1.176702
 5   0.764    P  24.568397    1.032   0.268  23.349227   1.219171
 7   1.032    P  24.736694    1.288   0.256  23.534150   1.202544
 9   1.288    P  24.576155    1.562   0.274  23.202312   1.373843
 11  1.562    P  24.280976    1.848   0.286  22

In [ ]:
# Pull trial finbeat data
fb_test_data = finbeats['Bass1S08']['fb_data']
fb_peak_params = pd.DataFrame()
fb_trough_params = pd.DataFrame()
print(fb_test_data)

peaksmask = fb_test_data.loc[:,'type']=='P'
fb_peak_params = fb_test_data.loc[peaksmask]
fb_peak_params['endtime'] = np.nan
fb_peak_params['period'] = np.nan
fb_peak_params['nxttrough'] = np.nan
fb_peak_params['amplitude'] = np.nan

for i in fb_peak_params.index.values:
    if i+2 in fb_peak_params.index.values:
        fb_peak_params.loc[i,'endtime'] = fb_peak_params.loc[i+2,'time']
        fb_peak_params.loc[i, 'period'] = fb_peak_params.loc[i,'endtime'] - fb_peak_params.loc[i,'time']
    else:
        fb_peak_params.loc[i,'endtime'] = np.nan
        fb_peak_params.loc[i, 'period'] = np.nan
    
    if i+1 in fb_test_data.index.values:
        fb_peak_params.loc[i,'nxttrough'] = fb_test_data.loc[i+1,'ypos']
        fb_peak_params.loc[i, 'amplitude'] = abs(fb_peak_params.loc[i,'nxttrough'] - fb_peak_params.loc[i,'ypos'])
    else: 
        fb_peak_params.loc[i,'nxttrough'] = np.nan
        fb_peak_params.loc[i,'amplitude'] = np.nan
        
print(fb_peak_params)

troughsmask = fb_test_data.loc[:,'type']=='T'
fb_trough_params = fb_test_data.loc[troughsmask]
fb_trough_params['endtime'] = np.nan
fb_trough_params['period'] = np.nan
fb_trough_params['nxtpeak'] = np.nan
fb_trough_params['amplitude'] = np.nan

for i in fb_trough_params.index.values:
    if i+2 in fb_trough_params.index.values:
        fb_trough_params.loc[i,'endtime'] = fb_trough_params.loc[i+2,'time']
        fb_trough_params.loc[i, 'period'] = fb_trough_params.loc[i,'endtime'] - fb_trough_params.loc[i,'time']
    else:
        fb_trough_params.loc[i,'endtime'] = np.nan
        fb_trough_params.loc[i, 'period'] = np.nan
    
    if i+1 in fb_test_data.index.values:
        fb_trough_params.loc[i,'nxtpeak'] = fb_test_data.loc[i+1,'ypos']
        fb_trough_params.loc[i,'amplitude'] = fb_trough_params.loc[i,'nxtpeak'] - fb_trough_params.loc[i,'ypos']
    else: 
        fb_trough_params.loc[i,'nxtpeak'] = np.nan
        fb_trough_params.loc[i,'amplitude'] = np.nan
        
print(fb_trough_params)

In [ ]:
tracklist_subset = ['Bass1S12', 'Bass1S09', 'Bass1S06']


for trial in tracklist_subset:
    data = tracklist[trial]['data']['pt2y_smth']
    length = data.size
    FR = tracklist[trial]['FPS']
    sampling = 1.0/FR
    x = pylab.arange(0, 1, 1. / length)

    cA = []
    cD = []

    (cA, cD) = pywt.dwt(data,'sym20')

    pylab.title(trial)
    pylab.imshow((cA,cD), extent=[-1, 1, 1, 31], cmap='bone', aspect='auto') 
    pylab.show()


